# Novelty Generation for Writer Transcripts

Consume existing media, manipulate to produce new novel media.


In [2]:
import copy 
import os
import subprocess
import json
import cv2
import sys
import numpy as np
import random
from tools.generation_operations import external_operation

JSON_FILE_LOCATIONS = "../generation_operations"

# WHERE ARE THE ROOT IMAGE AND RESULTS GOING
ROOT = '/data/iam_data/'

In [3]:
TEXTURE_IMAGES = '../../data/transcripts/background_images'

In [4]:

background_img_loc=[os.path.join(TEXTURE_IMAGES,f) for f in os.listdir(TEXTURE_IMAGES) if f.endswith('jpg')]
                   
background_img_val = {}
for item in background_img_loc:
    background_img_val[os.path.basename(item)] = cv2.imread(item, cv2.IMREAD_UNCHANGED)

In [5]:

def letter_image_op(text_image, bg_image_id='antique_bg.jpg'):
    #set offsets to random part of background
    bg_image = background_img_val[bg_image_id]
  
    if bg_image.shape[1] >= text_image.shape[1]:
        x_offset = int(random.random() * (bg_image.shape[1] - text_image.shape[1]))
    else:
        x_offset = 0 
        
    if bg_image.shape[0] >= text_image.shape[0]:
        y_offset = int(random.random() * (bg_image.shape[0] - text_image.shape[0]))
    else:
        y_offset = 0
        
    bg_image_select = bg_image[y_offset:y_offset+text_image.shape[0],
                               x_offset:x_offset+text_image.shape[1],:]

    if text_image.shape[0:2] != bg_image.shape[0:2]:
        bg_image_select= cv2.resize(bg_image_select,(text_image.shape[1],text_image.shape[0]))
        
    blended = np.copy(text_image)
    img_max = np.max(text_image,axis=2)
    for i in range(text_image.shape[2]):
        blended[img_max<250,i] = bg_image_select[img_max<250,i]
    return blended

def bg_image_op(text_image, bg_image_id='antique_bg.jpg', approach='minimum', color=[255,255,255]):
    #set offsets to random part of background
    bg_image = background_img_val[bg_image_id]
  
    if bg_image.shape[1] >= text_image.shape[1]:
        x_offset = int(random.random() * (bg_image.shape[1] - text_image.shape[1]))
    else:
        x_offset = 0 
        
    if bg_image.shape[0] >= text_image.shape[0]:
        y_offset = int(random.random() * (bg_image.shape[0] - text_image.shape[0]))
    else:
        y_offset = 0
        
    bg_image_select = bg_image[y_offset:y_offset+text_image.shape[0],
                               x_offset:x_offset+text_image.shape[1],:]

    if text_image.shape[0:2] != bg_image.shape[0:2]:
        bg_image_select= cv2.resize(bg_image_select,(text_image.shape[1],text_image.shape[0]))
        
    blended = np.copy(text_image)
    for i in range(text_image.shape[2]):
        if approach == "minimum":
            blended[:,:,i] = np.minimum(text_image[:,:,i],bg_image_select[:,:,i])
        elif approach == "factor":
            img = np.copy(text_image[:,:,i])
            # darker is letters
            factor = (255-img)/255.0
            blended[:,:,i] = bg_image_select[:,:,i]*(1-factor) + img*factor
        else:
            img = np.copy(text_image[:,:,i])
            blended[:,:,i] = bg_image_select[:,:,i]
            blended[:,:,i][img<250] = color[i]*((255-img)/255.0).astype(np.uint8)[img<250]
    return blended

def lines_color(image, color=[255,255,255]):
    img_max = np.max(image,axis=2)
    adjusted = np.copy(image)
    adjusted[img_max<250,:] = color
    return adjusted

def bg_color(image, color=[255,255,255]):
    img_min = np.max(image,axis=2)
    adjusted = np.copy(image)
    adjusted[img_min>250,:] = color
    return adjusted

def invert_image(image):
    return 255-image

def pen_clean(image):
    #i = np.copy(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    th, threshed = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
    threshed = cv2.cvtColor(threshed, cv2.COLOR_GRAY2BGR)
    threshed_marker = threshed>0
    new_img =  (image*threshed_marker) + (255 - threshed)
    s =np.logical_and(new_img > 1,new_img < 254)
    new_img[s] = new_img[s] - 1
    return new_img
        #new_loc = os.path.join(dest_dir,image[len(img_dir)+1:])
   # for c in range(3):
      #  i[:,:,c] = (i[:,:,c]>175)*255
   # return i

def pen_erode(image, kernel = np.ones((3,3),np.uint8)):
    return 255 - cv2.erode(255-image, kernel, iterations = 1)
    
def pen_dilate(image,kernel = np.ones((5,5),np.uint8)):
    if image is None:
        return None
    image = cv2.dilate(255-image, kernel, iterations = 1)
    return 255 - cv2.blur(cv2.blur(image, kernel.shape),kernel.shape)
   
def pen_smooth(image,kernel = np.ones((5,5),np.uint8)):
    #image = pen_dilate(image)
    return (255 - cv2.GaussianBlur(255 - image, kernel.shape, 0.5))

def gaussian_noise(image, var=0.5):
    row,col,ch= image.shape
    mean = 0
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))*10
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy.astype(np.uint8)

def flip(image):
    return cv2.flip(image, 0)

def slant(image, forward = True):
    #print(image.shape)
    newimage = np.ones(image.shape,dtype=np.uint8)*255
    try:
        smallimage = cv2.resize(image,(image.shape[1]-newimage.shape[0]*2,image.shape[0]))
    except Exception as ex:
        print(ex)
        return None
    image = np.ones(image.shape,dtype=np.uint8)*255
    image[:,newimage.shape[0]:-newimage.shape[0],:]  = smallimage
    for row in range(newimage.shape[0]):
        #print(newimage.shape)
        #print(flip_factor-row)
        if forward:
            newimage[row,:(-row if row > 0 else newimage.shape[1]),:] = image[row,row:,:]
        else:
            section = image[row,newimage.shape[0]-row-1:,:] 
            newimage[row,:section.shape[0],:] = section
    return newimage

def scale(image, transform_matrix = np.asarray([[1.5,1.5,0],[0,1,0,],[0,0,1]]).astype(float)):
    return cv2.warpPerspective(image, 
                                    transform_matrix, 
                                    (image.shape[1], image.shape[0]), 
                                    flags=cv2.INTER_LINEAR,
                                    borderMode=cv2.BORDER_CONSTANT, 
                                    borderValue=[255,255,255])

def pen_slant_dilate(image, forward=True):
    return pen_dilate(slant(image,forward=forward))

In [42]:
import glob
import datetime
from multiprocessing import Pool
PROCESSES = 6

def _run_one(source_dir, dest, filename, op, op_args):
    input_img = os.path.join(source_dir, filename)
    output_img_fn = os.path.join(dest, input_img[len(source_dir)+1:])
    input_img = os.path.join(source_dir, filename)
    os.makedirs(os.path.dirname(output_img_fn), exist_ok=True)
    print(f'{output_img_fn}')
    try:
        out_img = op(cv2.imread(input_img),**op_args)
        print(np.histogram(out_img))
        plt.imshow(out_img)
        if out_img is not None:
            cv2.imwrite(output_img_fn, out_img) 
    except Exception as ex:
            print(f'Error {str(ex)} on {output_img_fn}')
 
def _run_one_mp(input_img_fn, output_img_fn, op, op_args, i, since, skip_existing):
    try:
        if since is not None:
            mtime = os.stat(input_img_fn).st_mtime
            mtime = datetime.datetime.fromtimestamp(mtime)
            if mtime.day<since:
                    return None   
        if os.path.exists(output_img_fn) and skip_existing:
            out_img = cv2.imread(output_img_fn)
            if out_img is not None and out_img.shape[0] >= 10 and out_img.shape[1] >= 10:
                return None
        if i%500 == 0:
            print(f'{i}, {output_img_fn}')
        out_img = op(cv2.imread(input_img_fn),**op_args)
        if out_img is not None:
            cv2.imwrite(output_img_fn, out_img) 
        else:
            print('empty image {output_img_fn}')
            return None
        return output_img_fn
    except Exception as ex:
         print(f'Error {str(ex)} on {output_img_fn}')

def _run_all(source_dir, dest, op, op_args, since=None, skip_existing=True):
    images = glob.glob(os.path.join(source_dir, "**", "*.png"), recursive=True)
    print(f'running {len(images)} images ')
    completed = 0
    skipped = 0
    with Pool(PROCESSES) as pool:
        for i,input_img in enumerate(images):
            output_img_fn = os.path.join(dest, input_img[len(source_dir)+1:])
            pool.apply(_run_one_mp, (input_img, output_img_fn, op, op_args, i, since, skip_existing))
            completed+=1
    print(f'Completed {completed}; Skippped = {skipped}')
    
def run_choice(dest_dir, source_dir, choice, op, op_args, directory_postfix = '', 
               filename=None,
               skip_existing= True,
               since=None):
    dest = dest_dir + choice + directory_postfix
    if not os.path.exists(dest):
        os.mkdir(dest)
    if filename is not None:
        return _run_one(source_dir, dest, filename, op, op_args)
    _run_all(source_dir, dest, op, op_args, since=since, skip_existing=skip_existing)


In [43]:


dir_pairs = {
             'bg_blue_wall_red_letter': (bg_image_op, {'bg_image_id':'antique_bg.jpg', 'approach': 'overwrite', 'color': (30,40,200) }), 
             'bg_antique': (bg_image_op, {'bg_image_id':'antique_bg.jpg'}), 
             'bg_aged_paper_brown': (bg_image_op, {'bg_image_id':'bg_aged_paper_brown.jpg'}),
             'letter_aged_paper_brown': (letter_image_op, {'bg_image_id':'bg_aged_paper_brown.jpg'}),
             'bg_black_wall': (bg_image_op, {'bg_image_id':'bg_black_wall.jpg'}),
             'letter_black_wall': (letter_image_op, {'bg_image_id':'bg_black_wall.jpg'}),
             'bg_blue_cracked_wall': (bg_image_op, {'bg_image_id':'bg_blue_cracked_wall.jpg'}),
             'letter_blue_cracked_wall': (letter_image_op, {'bg_image_id':'bg_blue_cracked_wall.jpg'}),
             'bg_concrete_blocks': (bg_image_op, {'bg_image_id':'bg_concrete_blocks.jpg'}),
             'letter_concrete_blocks': (letter_image_op, {'bg_image_id':'bg_concrete_blocks.jpg'}),
             'bg_cracked_concrete': (bg_image_op, {'bg_image_id':'bg_cracked_concrete.jpg'}),
             'letter_cracked_concrete': (letter_image_op, {'bg_image_id':'bg_cracked_concrete.jpg'}),
             'bg_finished_wood': (bg_image_op, {'bg_image_id':'bg_finished_wood.jpg'}),
             'letter_finished_wood': (letter_image_op, {'bg_image_id':'bg_finished_wood.jpg'}),
             'bg_gray_concrete': (bg_image_op, {'bg_image_id':'bg_gray_concrete.jpg'}),
             'letter_gray_concrete': (letter_image_op, {'bg_image_id':'bg_gray_concrete.jpg'}),
             'bg_gray_crumpled': (bg_image_op, {'bg_image_id':'bg_gray_crumpled.jpg'}),
             'letter_gray_crumpled': (letter_image_op, {'bg_image_id':'bg_gray_crumpled.jpg'}),
             'bg_gray_wall': (bg_image_op, {'bg_image_id':'bg_gray_wall.jpg'}),
             'letter_gray_wall': (letter_image_op, {'bg_image_id':'bg_gray_wall.jpg'}),
             'bg_horizontal_cardboard': (bg_image_op, {'bg_image_id':'bg_horizontal_cardboard.jpg'}),
             'letter_horizontal_cardboard': (letter_image_op, {'bg_image_id':'bg_horizontal_cardboard.jpg'}),
             'bg_mixed_wood': (bg_image_op, {'bg_image_id':'bg_mixed_wood.jpg'}),
             'letter_mixed_wood': (letter_image_op, {'bg_image_id':'bg_mixed_wood.jpg'}),
             'bg_old_crumpled': (bg_image_op, {'bg_image_id':'bg_old_crumpled.jpg'}),
             'letter_old_crumpled': (letter_image_op, {'bg_image_id':'bg_old_crumpled.jpg'}),
             'bg_old_folder': (bg_image_op, {'bg_image_id':'bg_old_folder.jpg'}),
             'letter_old_folder': (letter_image_op, {'bg_image_id':'bg_old_folder.jpg'}),
             'bg_old_paper': (bg_image_op, {'bg_image_id':'bg_old_paper.jpg'}),
             'letter_old_paper': (letter_image_op, {'bg_image_id':'bg_old_paper.jpg'}),
             'bg_red_brick': (bg_image_op, {'bg_image_id':'bg_red_brick.jpg'}),
             'letter_red_brick': (letter_image_op, {'bg_image_id':'bg_red_brick.jpg'}),
             'bg_red_cracked_wall': (bg_image_op, {'bg_image_id':'bg_red_cracked_wall.jpg'}),
             'letter_red_cracked_wall': (letter_image_op, {'bg_image_id':'bg_red_cracked_wall.jpg'}),
             'bg_rust_stained_wall': (bg_image_op, {'bg_image_id':'bg_rust_stained_wall.jpg'}),
             'letter_rust_stained_wall': (letter_image_op, {'bg_image_id':'bg_rust_stained_wall.jpg'}),
             'bg_shadowed_white_wall': (bg_image_op, {'bg_image_id':'bg_shadowed_white_wall.jpg'}),
             'letter_shadowed_white_wall': (letter_image_op, {'bg_image_id':'bg_shadowed_white_wall.jpg'}),
             'bg_stained_paper': (bg_image_op, {'bg_image_id':'bg_stained_paper.jpg'}),
             'letter_stained_paper': (letter_image_op, {'bg_image_id':'bg_stained_paper.jpg'}),
             'bg_vertical_cardboard': (bg_image_op, {'bg_image_id':'bg_vertical_cardboard.jpg'}),
             'letter_vertical_cardboard': (letter_image_op, {'bg_image_id':'bg_vertical_cardboard.jpg'}),
             'bg_white_creased_paper': (bg_image_op, {'bg_image_id':'bg_white_creased_paper.jpg'}),
             'letter_white_creased_paper': (letter_image_op, {'bg_image_id':'bg_white_creased_paper.jpg'}),
             'bg_white_crumpled_paper': (bg_image_op, {'bg_image_id':'bg_white_crumpled_paper.jpg'}),
             'letter_white_crumpled_paper': (letter_image_op, {'bg_image_id':'bg_white_crumpled_paper.jpg'}),
             'bg_yellow_cracked_wall': (bg_image_op, {'bg_image_id':'bg_yellow_cracked_wall.jpg'}),
             'letter_yellow_cracked_wall': (letter_image_op, {'bg_image_id':'bg_yellow_cracked_wall.jpg'}),
             'bg_crinked_paper': (bg_image_op, {'bg_image_id':'bg_crinked_paper.jpg'}),
             'bg_blue_wall': (bg_image_op, {'bg_image_id':'bg_blue_wall.jpg'}),
             'bg_gold_texture_wall': (bg_image_op, {'bg_image_id':'bg_gold_texture_wall.jpg'}),
             'bg_brown_texture_fabric': (bg_image_op, {'bg_image_id':'bg_brown_texture_fabric.jpg'}),
             'bg_rock': (bg_image_op, {'bg_image_id':'bg_rock.jpg'}),
             'bg_rough_paper': (bg_image_op, {'bg_image_id':'bg_rough_paper.jpg'}),
             'letter_gold_texture':(letter_image_op,{'bg_image_id':'bg_gold_texture_wall.jpg'}),
             'letter_brown_texture':(letter_image_op,{'bg_image_id':'bg_brown_texture_fabric.jpg'}),
             'letter_rainbow':(letter_image_op,{'bg_image_id':'bg_rainbow_paper.jpg'}),
             'bg_rainbow_paper':(bg_image_op,{'bg_image_id':'bg_rainbow_paper.jpg'}),
             'bg_gaussian_noise': (gaussian_noise, {'var': 0.5}),
             'letter_inverted': (invert_image,{}),
             'bg_blue_color': (bg_color,{'color': (200,30,40)}),
             'letter_blue': (lines_color,{'color': (200,30,40)}),
             'letter_red': (lines_color,{'color': (30,40,200)}),
             'letter_dilate': (pen_dilate, {}),
             'letter_erode': (pen_erode, {}),
             'letter_flip': (flip, {}),
             'letter_scale': (scale, { 'transform_matrix': np.asarray([[1.5,1.5,0],[0,1,0,],[0,0,1]]).astype(float)}),
             'letter_slant_small': (scale, { 'transform_matrix': np.asarray([[0.5,0.5,0],[0,0.75,0,],[0,0,1]]).astype(float)}),
             'letter_slant_big': (slant,{}),
             'letter_slant_left_big': (slant,{'forward': False}),
             'letter_slant_dilate': (pen_slant_dilate, {}),
             'letter_clean': (pen_clean,{})
            }



In [ ]:
# pick the choices or set to [] for running all possible choices

# this runs first
# set IMAGE_DIR = ROOT + f'{SUFFIX}' before using

SUFFIX = '-lines_generated'

seeding_m18 = ['letter_clean']
# WHICH IMAGE SET TO AUGMENT

choices_bg_m18 = ['bg_blue_cracked_wall',
'bg_brown_texture_fabric',
'bg_concrete_blocks',
'bg_crinked_paper',
'bg_gold_texture_wall',
'bg_horizontal_cardboard',
'bg_rainbow_paper',
'bg_rust_stained_wall',
'bg_yellow_cracked_wall',
'bg_old_paper']

choices_letter_18 = ['letter_blue',
'letter_brown_texture',
'letter_dilate',
'letter_flip',
'letter_gold_texture',
'letter_inverted',
'letter_old_paper',
'letter_rainbow',
'letter_red',
'letter_scale',
'letter_slant_big',
'letter_slant_dilate',
'letter_slant_left_big',
'letter_slant_small',
'letter_yellow_cracked_wall']


choices = seeding_m18

if choices[0] == seeding_m18[0]:
    IMAGE_DIR = ROOT + 'lines_generated'
else:
    IMAGE_DIR = ROOT + f'letter_clean{SUFFIX}'
if len(choices) > 0:
    for choice in choices:
        run_choice(ROOT,IMAGE_DIR, choice, 
                   dir_pairs[choice][0], dir_pairs[choice][1], 
                   directory_postfix=SUFFIX,
                   #filename='p01-147-589-65.png',
                   since=None,
                   skip_existing=False) 